# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    

In [1]:
from __future__ import division
import numpy as np
import sframe
import pandas as pd
import math
import string

# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = sframe.SFrame('amazon_baby.gl/')

[INFO] sframe.cython.cy_server: SFrame v2.1 started. Logging /tmp/sframe_server_1519166054.log


Now, let us see a preview of what the dataset looks like.

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [4]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [5]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

products['review_clean'] = products['review'].apply(remove_punctuation)
#products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [6]:
#products[269]['word_count']

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [7]:
products = products[products['rating'] != 3]
len(products)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [8]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,review_clean,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,it came early and was notdisappointed i love ...,1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,Very soft and comfortableand warmer than it ...,1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,This is a product wellworth the purchase I ...,1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,All of my kids have criednonstop when I tried to ...,1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,When the Binky Fairy cameto our house we didnt ...,1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,Lovely book its boundtightly so you may no ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,Perfect for new parentsWe were able to keep ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,A friend of mine pinnedthis product on Pinte ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,This has been an easy wayfor my nanny to record ...,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,I love this journal andour nanny uses it ...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [9]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


## Build the word count vector for each review
 We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

1. Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
2. Compute the occurrences of the words in each review and collect them into a row vector.
3. Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
4. Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.



In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
 # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [11]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression(n_jobs=-1, verbose=1)
sentiment_model.fit(train_matrix, train_data['sentiment'])

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=1, warm_start=False)

In [12]:
sentiment_model

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=1, warm_start=False)

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

In [13]:
feature_names = ['intercept']+vectorizer.vocabulary_.keys()
feature_values = np.hstack((sentiment_model.intercept_, sentiment_model.coef_[0]))

In [14]:
len(feature_names)

121713

In [15]:
len(feature_values)

121713

In [16]:
weights=pd.DataFrame({'value': feature_values, 'name': feature_names})

In [17]:
weights.columns

Index([u'name', u'value'], dtype='object')

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [18]:
num_positive_weights = len(weights[weights['value']>=0])
num_negative_weights = len(weights[weights['value']<0])

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 87244 
Number of negative weights: 34469 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [19]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


name,review,rating,review_clean,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,Absolutely love it andall of the Scripture in ...,1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,Would not purchase againor recommend The decals ...,-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,Was so excited to getthis product for my baby ...,-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [20]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [21]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores**. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.  In scikit-learn, you can call the decision_function() function.

In [22]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60153832  -3.17045551 -10.42328017]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [23]:
yhat = [1 if score >0 else -1 for score in scores]
yhat

[1, -1, -1]

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [24]:
print "Class predictions according to Scikit Learn:" 
print sentiment_model.predict(sample_test_matrix)

Class predictions according to Scikit Learn:
[ 1 -1 -1]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [25]:
P = [1/(1+math.exp(-score))  for score in scores]
P

[0.9963214024609771, 0.040292797410969486, 2.973130874674057e-05]

In [26]:
print "Class predictions according to scikit learn:" 
print sentiment_model.predict_proba(sample_test_matrix)

Class predictions according to scikit learn:
[[  3.67859754e-03   9.96321402e-01]
 [  9.59707203e-01   4.02927974e-02]
 [  9.99970269e-01   2.97313087e-05]]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [27]:
test_data['probability'] = sentiment_model.predict_proba(test_matrix)[:,1]

In [28]:
sentiment_model.predict_proba(test_matrix)

array([[  2.15496740e-01,   7.84503260e-01],
       [  7.61209113e-07,   9.99999239e-01],
       [  6.68077417e-02,   9.33192258e-01],
       ..., 
       [  5.16538748e-06,   9.99994835e-01],
       [  2.41151057e-06,   9.99997588e-01],
       [  1.82497531e-02,   9.81750247e-01]])

In [29]:
sentiment_model.predict_proba(test_matrix)[:,1]

array([ 0.78450326,  0.99999924,  0.93319226, ...,  0.99999483,
        0.99999759,  0.98175025])

In [40]:
probability = list(sentiment_model.predict_proba(test_matrix)[:,1])
probability


[0.78450326024795936,
 0.99999923879088726,
 0.93319225831486974,
 0.99997945670069466,
 0.98023052196864524,
 0.99995879927141373,
 0.9987566526116427,
 0.80134213047486313,
 0.99844590084429041,
 0.99737547406282301,
 0.99632140246097711,
 0.040292797410969486,
 2.973130874674057e-05,
 0.99578705939400169,
 0.99299147996506187,
 0.9770715253000164,
 0.99272128039572671,
 0.94003383375582927,
 0.039839416116966092,
 0.99897747965974504,
 0.99863703458120512,
 0.99907457414027545,
 0.99999995131270658,
 0.99996979029611777,
 0.99997639856777176,
 0.97896713147205661,
 0.97525017035980099,
 0.95307299555320446,
 0.99999999964140929,
 0.99963914560631006,
 0.99039567562459596,
 0.99960820626992131,
 0.99995978178896483,
 0.99999868322061025,
 0.99998824681894249,
 0.99940716183709077,
 0.99985837703462677,
 0.99518965457318398,
 0.92418487650470982,
 0.9944203054572982,
 0.81345813723490312,
 0.99243296771986256,
 0.92573688624115624,
 0.97972522209909085,
 0.98379752362360129,
 0.996819

In [41]:
sorted(probability, reverse=True)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.99999999999999978,
 0.99999999999999978,
 0.99999999999999956,
 0.99999999999999956,
 0.99999999999999933,
 0.99999999999999822,
 0.999999999999998,
 0.999999999999994,
 0.99999999999999245,
 0.99999999999998956,
 0.99999999999998801,
 0.9999999999999809,
 0.99999999999997313,
 0.99999999999997247,
 0.99999999999997136,
 0.99999999999996558,
 0.99999999999996558,
 0.99999999999996492,
 0.99999999999996403,
 0.99999999999995692,
 0.9999999999999527,
 0.99999999999993805,
 0.99999999999991895,
 0.99999999999991696,
 0.99999999999988143,
 0.99999999999987321,
 0.99999999999986922,
 0.99999999999986477,
 0.99999999999985678,
 0.99999999999985278,
 0.99999999999975464,
 0.99999999999975353,
 0.99999999999974731,
 0.99999999999973799,
 0.99999999999972511,
 0.99999999999969313,
 0.99999999999967137,
 0.99999999999966738,
 0.99999999999966604,
 0.99999999999957923,
 0.99999999999957367,
 0

In [42]:
sorted(probability)

[8.4388311068173374e-16,
 1.6031942106676011e-15,
 8.166342247089814e-14,
 9.849655568704647e-14,
 1.9256602963696026e-13,
 3.3277598845370015e-13,
 3.2747468520117969e-11,
 3.3217342121688498e-11,
 9.47743149178608e-11,
 9.6139557147587839e-11,
 4.3561002221427455e-10,
 4.3916438833182571e-10,
 5.5787073655895455e-10,
 5.6898609208801001e-10,
 5.8355865783182605e-10,
 6.1869201447828441e-10,
 8.052691452625961e-10,
 1.0781255839382047e-09,
 1.5938825865273517e-09,
 1.652043445396831e-09,
 1.832500279568269e-09,
 1.8990988666679386e-09,
 3.1642018435356241e-09,
 3.1898768957895415e-09,
 3.1965506945563983e-09,
 3.4433644584265441e-09,
 4.8268212117389701e-09,
 5.2821201185575034e-09,
 6.6942249718379735e-09,
 8.3662082935096966e-09,
 9.40668105330697e-09,
 9.9296012763879083e-09,
 1.2245424320697966e-08,
 1.3476884134764734e-08,
 1.7536873492149141e-08,
 2.4089288098647459e-08,
 2.6642820845072501e-08,
 3.3633755724048223e-08,
 3.7030945170353865e-08,
 3.9161291415011283e-08,
 3.988842

In [45]:
sorted(probability)[:20]

[8.4388311068173374e-16,
 1.6031942106676011e-15,
 8.166342247089814e-14,
 9.849655568704647e-14,
 1.9256602963696026e-13,
 3.3277598845370015e-13,
 3.2747468520117969e-11,
 3.3217342121688498e-11,
 9.47743149178608e-11,
 9.6139557147587839e-11,
 4.3561002221427455e-10,
 4.3916438833182571e-10,
 5.5787073655895455e-10,
 5.6898609208801001e-10,
 5.8355865783182605e-10,
 6.1869201447828441e-10,
 8.052691452625961e-10,
 1.0781255839382047e-09,
 1.5938825865273517e-09,
 1.652043445396831e-09]

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [30]:
test_data[test_data['probability']==1.0]['name']

dtype: str
Rows: ?
['Britax Decathlon Convertible Car Seat, Tiffany', "P'Kolino Silly Soft Seating in Tias, Green", 'Evenflo X Sport Plus Convenience Stroller - Christina', 'Evenflo 6 Pack Classic Glass Bottle, 4-Ounce', 'Simple Wishes Hands-Free Breastpump Bra, Pink, XS-L', 'Baby Einstein Around The World Discovery Center', 'Freemie Hands-Free Concealable Breast Pump Collection System', 'Infantino Wrap and Tie Baby Carrier, Black Blueberries', "Fisher-Price Cradle 'N Swing,  My Little Snugabunny", 'Roan Rocco Classic Pram Stroller 2-in-1 with Bassinet and Seat Unit - Coffee', 'Britax 2012 B-Agile Stroller, Red', "Graco Pack 'n Play Element Playard - Flint", 'Diono RadianRXT Convertible Car Seat, Plum', 'Baby Jogger City Mini GT Single Stroller, Shadow/Orange', 'Buttons Cloth Diaper Cover - One Size - 8 Color Options', 'Graco FastAction Fold Jogger Click Connect Stroller, Grapeade', 'Mamas &amp; Papas 2014 Urbo2 Stroller - Black', ... ]

In [43]:
test_data[test_data['probability']==8.4388311068173374e-16]['name']

dtype: str
Rows: ?
['Fisher-Price Ocean Wonders Aquarium Bouncer', ... ]

In [44]:
test_data[test_data['probability']==1.6031942106676011e-15]['name']

dtype: str
Rows: ?
["Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)", ... ]

In [46]:
[test_data[test_data['probability']==a]['name'] for a in sorted(probability)[:20]]

[dtype: str
 Rows: ?
 ['Fisher-Price Ocean Wonders Aquarium Bouncer', ... ], dtype: str
 Rows: ?
 ["Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)", ... ], dtype: str
 Rows: ?
 ['Safety 1st Exchangeable Tip 3 in 1 Thermometer', ... ], dtype: str
 Rows: ?
 ['Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow Flow (0-3 months)', ... ], dtype: str
 Rows: ?
 ['VTech Communications Safe &amp; Sounds Full Color Video and Audio Monitor', ... ], dtype: str
 Rows: ?
 ['The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit', ... ], dtype: str
 Rows: ?
 ['Safety 1st High-Def Digital Monitor', ... ], dtype: str
 Rows: ?
 ['Cloth Diaper Sprayer--styles may vary', ... ], dtype: str
 Rows: ?
 ['Philips AVENT Newborn Starter Set', ... ], dtype: str
 Rows: ?
 ['Motorola Digital Video Baby Monitor with Room Temperature Thermometer', ... ], dtype: str
 Rows: ?
 ['Ellaroo Mei Tai Baby Carrier - Hershey', ... ], dtype:

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [47]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print scores

[  5.60153832  -3.17045551 -10.42328017]


In [48]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    matrix = vectorizer.transform(data['review_clean'])
    predictions = model.predict(matrix)
    
    # Compute the number of correctly classified examples
    correct = sum(predictions==true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    accuracy = correct/len(true_labels)
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [49]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.93229541636669067

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [50]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [51]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [52]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

Let's see what the first example of the dataset looks like:

In [53]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [54]:
simple_model = LogisticRegression(n_jobs=-1, verbose=1)
simple_model.fit(train_matrix_word_subset, train_data['sentiment'])
simple_model

[LibLinear]

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=1, warm_start=False)

In [55]:
def get_classification_accuracy2(model, matrix, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(matrix)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correct = sum(predictions==true_labels)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correct/len(true_labels)
    return accuracy

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [56]:
get_classification_accuracy2(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.86936045116390692

In [57]:
simple_model_coef_table = pd.DataFrame({'word': significant_words,
                                        'coefficient': simple_model.coef_[0]})
simple_model_coef_table

,coefficient,word
0,1.363690,love
1,0.944000,great
2,1.192538,easy
3,0.085513,old
4,0.520186,little
5,1.509812,perfect
6,1.673074,loves
7,0.503760,well
8,0.190909,able
9,0.058855,car


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [58]:
simple_model_coef_table.sort_values('coefficient', ascending=False)

,coefficient,word
6,1.673074,loves
5,1.509812,perfect
0,1.363690,love
2,1.192538,easy
1,0.944000,great
4,0.520186,little
7,0.503760,well
8,0.190909,able
3,0.085513,old
9,0.058855,car


**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [59]:
simple_model_coef_table[simple_model_coef_table.coefficient>0].shape

(10, 2)

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [60]:
positive_significant_words = simple_model_coef_table[simple_model_coef_table.coefficient>0]['word']

In [61]:
positive_significant_words

0       love
1      great
2       easy
3        old
4     little
5    perfect
6      loves
7       well
8       able
9        car
Name: word, dtype: object

In [62]:
for word in positive_significant_words:
    if word in weights[weights.value>0]['name'].values:
        print 'true'

true
true
true
true
true
true


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [63]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

0.96848953648737779

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [64]:
get_classification_accuracy2(simple_model, train_matrix_word_subset, train_data['sentiment'])

0.8668225700065959

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [65]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.93229541636669067

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [66]:
get_classification_accuracy2(simple_model, test_matrix_word_subset, test_data['sentiment'])

0.86936045116390692

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [67]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112164
21252


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [68]:
accuracy_baseline = (test_data['sentiment'] == +1).sum()/len(test_data)
print accuracy_baseline

0.842782577394


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?